## **Voorbeelden: Multi-AI Agents voor het boeken van een hotel**

In de snelle wereld van vandaag omvat het plannen van een zakenreis meer dan alleen het boeken van een vlucht en een hotelkamer. Het vereist een niveau van coördinatie en efficiëntie dat moeilijk te bereiken kan zijn. Hier komen Multi-AI Agents in beeld, die de manier waarop we onze reisbehoeften beheren revolutioneren.

Stel je voor dat je een team van intelligente agenten tot je beschikking hebt, die samenwerken om elk aspect van je reis nauwkeurig en moeiteloos af te handelen. Met onze geavanceerde AI-technologie hebben we gespecialiseerde agenten gecreëerd voor het boeken van diensten en het regelen van de reisroute, wat zorgt voor een naadloze en stressvrije reiservaring.

Dit is een basisscenario. Bij het plannen van een zakenreis moeten we contact opnemen met een zakenreisagent om informatie over vliegtickets, hotelinformatie, enz. te verkrijgen. Via AI Agents kunnen we agenten bouwen voor het boeken van diensten en agenten voor het regelen van de reisroute om samen te werken en het intelligentieniveau te verbeteren.


# Initialiseer de Azure AI Agent Service en haal configuratie-informatie op uit **.env**

### **.env**

Maak een .env-bestand aan

**.env** bevat de verbindingsreeks van Azure AI Agent Service, het model dat door AOAI wordt gebruikt, en de bijbehorende Google API Search service API, ENDPOINT, enz.

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "Uw Azure AI Agent Service Model Deployment Naam"

[**NOTE**] U heeft een model nodig met een Rate Limit van 100.000 (Tokens per minuut) en een Rate Limit van 600 (Verzoeken per minuut)

  U kunt het model verkrijgen in Microsoft Foundry - Model en Endpoint.

- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "Uw Azure AI Agent Service Project Verbindingstekenreeks"

  U kunt de projectverbindingstekenreeks vinden in uw projectoverzicht in het AI Foundry Portal scherm.

- **SERPAPI_SEARCH_API_KEY** = "Uw SERPAPI Search API SLEUTEL"
- **SERPAPI_SEARCH_ENDPOINT** = "Uw SERPAPI Search Endpoint"

Om de Model Deployment Naam en Project Verbindingstekenreeks van Azure AI Agent Service te verkrijgen, moet u Azure AI Agent Service aanmaken. Het wordt aanbevolen om [deze template](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json) direct te gebruiken om deze aan te maken (***Let op:*** Azure AI Agent Service wordt momenteel ingesteld in een beperkte regio. Het wordt aanbevolen om [deze link](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support) te raadplegen om de regio te configureren)

De agent heeft toegang nodig tot SERPAPI. Het wordt aanbevolen om u te registreren via [deze link](https://serpapi.com/searches). Na registratie kunt u een unieke API SLEUTEL en ENDPOINT verkrijgen.


# Inloggen bij Azure 

Je moet nu inloggen bij Azure. Open een terminal in VScode en voer het commando `az login` uit.


# Setup 

Om deze notebook uit te voeren, moet je de volgende bibliotheken installeren. Hier is een lijst van de benodigde bibliotheken en de bijbehorende pip install commando's:

azure-identity: Voor Azure authenticatie.  
requests: Voor het maken van HTTP-verzoeken.  
semantic-kernel: Voor het semantic kernel framework (aangezien dit een aangepaste of specifieke bibliotheek is, moet je deze mogelijk installeren vanuit een specifieke bron of repository).


In [ ]:
!pip install azure-identity
!pip install requests
!pip install semantic-kernel
!pip install --upgrade semantic_kernel
!pip install azure-cli

# Uitleg: 
import asyncio: Dit importeert de asyncio-module, die ondersteuning biedt voor asynchrone programmering in Python. Hiermee kun je gelijktijdige code schrijven met behulp van de async- en await-syntaxis.
from typing import Annotated: Dit importeert het type Annotated uit de typing-module. Annotated wordt gebruikt om metadata toe te voegen aan type-aanwijzingen, wat nuttig kan zijn voor verschillende doeleinden zoals validatie, documentatie of tooling.


In [ ]:
import asyncio,os
from typing import Annotated

# Uitleg:
Door gebruik te maken van from dotenv import load_dotenv en load_dotenv(), kunt u eenvoudig configuratie-instellingen en gevoelige informatie (zoals API-sleutels en database-URL's) beheren in een .env-bestand, waardoor deze gescheiden blijven van uw broncode en uw applicatie veiliger en gemakkelijker te configureren wordt.


In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Uitleg:

Importverklaring: from azure.identity.aio import DefaultAzureCredential: Dit importeert de DefaultAzureCredential-klasse uit de azure.identity.aio-module. Het aio-gedeelte van de modulenaam geeft aan dat het is ontworpen voor asynchrone operaties.

Doel van DefaultAzureCredential: De DefaultAzureCredential-klasse maakt deel uit van de Azure SDK voor Python. Het biedt een standaardmanier om te authenticeren bij Azure-diensten. Het probeert te authenticeren met meerdere methoden in een specifieke volgorde, zoals omgevingsvariabelen, beheerde identiteit en Azure CLI-referenties.

Asynchrone operaties: De aio-module geeft aan dat de DefaultAzureCredential-klasse asynchrone operaties ondersteunt. Dit betekent dat je het kunt gebruiken met asyncio om niet-blokkerende authenticatie-aanvragen uit te voeren.


In [ ]:
from azure.identity.aio import DefaultAzureCredential

# Uitleg:
Importeert verschillende modules en klassen uit het semantic_kernel-pakket. Hier is een overzicht van elke import:

AgentGroupChat uit semantic_kernel.agents: Deze klasse behandelt functionaliteiten gerelateerd aan groepschat voor AI-agenten. AzureAIAgent en AzureAIAgentSettings uit semantic_kernel.agents.azure_ai

AzureAIAgent: Deze klasse wordt gebruikt om AI-agenten te maken en beheren die gebruikmaken van Azure AI-diensten.

AzureAIAgentSettings: Deze klasse wordt gebruikt om instellingen voor de AzureAIAgent te configureren. TerminationStrategy uit semantic_kernel.agents.strategies.termination.termination_strategy:

Deze klasse definieert strategieën voor het beëindigen van de uitvoering van AI-agenten onder bepaalde voorwaarden. ChatMessageContent uit semantic_kernel.contents.chat_message_content:

Deze klasse wordt gebruikt om de inhoud van chatberichten te verwerken.
AuthorRole uit semantic_kernel.contents.utils.author_role:

Deze klasse definieert verschillende rollen voor auteurs in de context van chatberichten.

kernel_function uit semantic_kernel.functions.kernel_function_decorator: Deze decorator wordt gebruikt om kernel-functies te definiëren, dat zijn functies die binnen het semantic kernel-framework kunnen worden uitgevoerd.
Deze imports zetten de benodigde componenten op voor het creëren en beheren van AI-agenten die kunnen communiceren in een groepschatomgeving, mogelijk voor taken zoals het boeken van hotels of soortgelijke activiteiten.


In [ ]:
from semantic_kernel.agents import AgentGroupChat
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings
from semantic_kernel.agents.strategies.termination.termination_strategy import TerminationStrategy
from semantic_kernel.contents import ChatMessageContent
from semantic_kernel.contents import AuthorRole
from semantic_kernel.functions.kernel_function_decorator import kernel_function

# Uitleg:
Vervolgens importeren we de klasse CodeInterpreterTool uit de module azure.ai.projects.models.

CodeInterpreterTool: Deze klasse maakt deel uit van de Azure AI SDK en wordt gebruikt om code te interpreteren en uit te voeren binnen de context van AI-projecten. Het biedt functionaliteiten voor het uitvoeren van codefragmenten, het analyseren van code of het integreren van code-uitvoering binnen AI-workflows.
Deze import stelt de benodigde component in voor het gebruiken van de CodeInterpreterTool in je project, wat nuttig kan zijn voor taken die het dynamisch interpreteren en uitvoeren van code omvatten.


In [ ]:
from azure.ai.projects.models import CodeInterpreterTool

# Uitleg: 
De ApprovalTerminationStrategy-klasse biedt een specifieke strategie voor het beëindigen van de werking van een AI-agent. De agent zal beëindigen als het laatste bericht in zijn interactiegeschiedenis het woord "saved" bevat. Dit kan nuttig zijn in scenario's waarin de taak van de agent als voltooid wordt beschouwd zodra hij de bevestiging ontvangt dat iets is "saved".Definieer de interactiemethode. Nadat het reserveringsplan is opgeslagen, kan het worden gestopt wanneer het saved-signaal wordt ontvangen.


In [ ]:
class ApprovalTerminationStrategy(TerminationStrategy):
    """A strategy for determining when an agent should terminate."""

    async def should_agent_terminate(self, agent, history):
        """Check if the agent should terminate."""
        return "saved" in history[-1].content.lower()

# Uitleg:

De regel code initialiseert een AzureAIAgentSettings-object met standaard- of vooraf gedefinieerde instellingen door de create()-methode aan te roepen. Dit instellingenobject (ai_agent_settings) kan vervolgens worden gebruikt om een AzureAIAgent-instantie te configureren en beheren.


In [ ]:
ai_agent_settings = AzureAIAgentSettings.create()

# Uitleg:
Door de requests-bibliotheek te importeren, kun je eenvoudig HTTP-verzoeken doen en communiceren met webservices in je Python-code.


In [ ]:
import requests

# Uitleg:
Dit is een variabele die de API-sleutel opslaat voor toegang tot een SERP (Search Engine Results Page) API-service. Een API-sleutel is een unieke identificatiecode die wordt gebruikt om verzoeken te authenticeren die aan jouw account zijn gekoppeld.

'GOOGLE_SEARCH_API_KEY': Dit is een tijdelijke tekenreeks. Je moet ''GOOGLE_SEARCH_API_KEY' vervangen door jouw eigen SERP API-sleutel.

Doel: Het doel van deze regel is om de API-sleutel op te slaan in een variabele zodat deze kan worden gebruikt om verzoeken aan de SERP API-service te authenticeren. De API-sleutel is vereist om toegang te krijgen tot de service en zoekopdrachten uit te voeren.

Hoe krijg je een SERP API-sleutel: Volg deze algemene stappen op https://serpapi.com om een SERP API-sleutel te krijgen (de exacte stappen kunnen variëren afhankelijk van de specifieke SERP API-service die je gebruikt):

Kies een SERP API-service: Er zijn verschillende SERP API-services beschikbaar, zoals SerpAPI, Google Custom Search JSON API en anderen. Kies degene die het beste bij jouw behoeften past.

Meld je aan voor een account:

Ga naar de website van de gekozen SERP API-service https://www.serpapi.com en meld je aan voor een account. Mogelijk moet je wat basisinformatie opgeven en je e-mailadres verifiëren.

Maak een API-sleutel aan:

Log na het aanmelden in op je account en ga naar het API-gedeelte of dashboard. Zoek naar een optie om een nieuwe API-sleutel aan te maken of te genereren.
Kopieer de API-sleutel:

Zodra de API-sleutel is gegenereerd, kopieer je deze. Deze sleutel wordt gebruikt om je verzoeken aan de SERP API-service te authenticeren.
Vervang de tijdelijke aanduiding:

Vervang de tijdelijke aanduiding in je .env-bestand


In [ ]:
SERPAPI_SEARCH_API_KEY=os.getenv('SERPAPI_SEARCH_API_KEY')

In [ ]:
SERPAPI_SEARCH_ENDPOINT = os.getenv('SERPAPI_SEARCH_ENDPOINT')

# Uitleg:
De BookingPlugin klasse biedt methoden voor het boeken van hotels en vluchten met behulp van de Serpapi.com Google Search API. Het stelt de benodigde parameters samen, verstuurt API-aanvragen en verwerkt de reacties om relevante boekingsinformatie terug te geven. De API-sleutel (SERPAPI_SEARCH_API_KEY) en het eindpunt (SERPAPI_SEARCH_ENDPOINT) worden gebruikt om te authenticeren en aanvragen te versturen naar de Google Search API.


In [ ]:
# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""
    @kernel_function(description="booking hotel")
    def booking_hotel(self,query: Annotated[str, "The name of the city"], check_in_date: Annotated[str, "Hotel Check-in Time"], check_out_date: Annotated[str, "Hotel Check-in Time"])-> Annotated[str, "Return the result of booking hotel infomation"]:

        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "2",
            "currency": "USD",
            "gl": "us",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY
        }

        response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=params)
        if response.status_code == 200:
            response = response.json()
            return response["properties"]
        else:
            return None

    
    @kernel_function(description="booking fight")
    def  booking_fight(self,origin: Annotated[str, "The name of Departure"], destination: Annotated[str, "The name of Destination"], outbound_date: Annotated[str, "The date of outbound"], return_date: Annotated[str, "The date of Return_date"])-> Annotated[str, "Return the result of booking fight infomation"]:
        
        go_params = {
            "engine": "google_flights",   
            "departure_id": origin,
            "arrival_id": destination,
            "outbound_date": outbound_date,
            "return_date": return_date,  
            "currency": "USD",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY  
        }

        print(go_params)

        go_response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=go_params)


        result = ''

        if go_response.status_code == 200:
            response = go_response.json()

            result += "# outbound \n " + str(response)
        else:
            print('error!!!')
            # return None

        
        back_params = {
            "engine": "google_flights",   
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": return_date,
            "return_date": return_date,  
            "currency": "USD",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY  
        }


        print(back_params)


        back_response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=back_params)



        if back_response.status_code == 200:
            response = back_response.json()

            result += "\n # return \n"  + str(response)

        else:
            print('error!!!')
            # return None
        
        print(result)

        return result

        


# Uitleg:
De SavePlugin-klasse biedt een methode saving_plan om reisplannen op te slaan met behulp van Azure AI-services. Het stelt Azure-referenties in, maakt een AI-agent aan, verwerkt gebruikersinvoer om de inhoud van het reisplan te genereren en op te slaan, en behandelt het opslaan van bestanden en opruimoperaties. De methode geeft "Opgeslagen" terug bij succesvolle voltooiing.


In [ ]:
class SavePlugin:
    """Save Plugin for customers"""
    @kernel_function(description="saving plan")
    async def saving_plan(self,tripplan: Annotated[str, "The content of trip plan"])-> Annotated[str, "Return status of save content"]:

        async with (
            DefaultAzureCredential() as creds,
            AzureAIAgent.create_client(
                credential=creds,
                conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
            ) as client,
        ):

            code_interpreter = CodeInterpreterTool()
            
            agent_definition = await client.agents.create_agent(
                model=ai_agent_settings.model_deployment_name,
                tools=code_interpreter.definitions,
                tool_resources=code_interpreter.resources,
            )


            agent = AzureAIAgent(
                client=client,
                definition=agent_definition,
            )

            thread = await client.agents.create_thread()


            user_inputs = [
                """
            
                        You are my Python programming assistant. Generate code,save """+ tripplan +
                        
                    """    
                        and execute it according to the following requirements

                        1. Save blog content to trip-{YYMMDDHHMMSS}.md

                        2. give me the download this file link
                    """
            ]



            try:
                for user_input in user_inputs:
                    # Add the user input as a chat message
                    await agent.add_chat_message(
                        thread_id=thread.id, message=ChatMessageContent(role=AuthorRole.USER, content=user_input)
                    )
                    print(f"# User: '{user_input}'")
                    # Invoke the agent for the specified thread
                    async for content in agent.invoke(thread_id=thread.id):
                        if content.role != AuthorRole.TOOL:
                            print(f"# Agent: {content.content}")

                    
                    messages = await client.agents.list_messages(thread_id=thread.id)

                    # OpenAIPageableListOfThreadMessage
                    # OpenAIPageableListOfThreadMessage


                    for file_path_annotation in messages.file_path_annotations:

                            file_name = os.path.basename(file_path_annotation.text)

                            await client.agents.save_file(file_id=file_path_annotation.file_path.file_id, file_name=file_name,target_dir="./trip")

                    
            finally:
                await client.agents.delete_thread(thread.id)
                await client.agents.delete_agent(agent.id)


        return "Saved"

# Uitleg:
Deze code zet Azure AI-agenten op om het boeken van vluchten en hotels te regelen en reisplannen op te slaan op basis van gebruikersinvoer. Het gebruikt Azure-referenties om de agenten te creëren en configureren, verwerkt gebruikersinvoer via een groepschat en zorgt voor correcte opruiming nadat de taken zijn voltooid. De agenten gebruiken specifieke plugins (BookingPlugin en SavePlugin) om hun respectieve taken uit te voeren.


In [ ]:
async with (
    DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
    ) as client,
):
    BOOKING_AGENT_NAME = "BookingAgent"
    BOOKING_AGENT_INSTRUCTIONS = """
    You are a booking agent. Help me book flights or hotels.

    Thought: Please understand the user's intention and confirm whether to use the reservation system to complete the task.

    Actions:
    - For flight bookings, convert the departure and destination names into airport codes.
    - Use the appropriate API for hotel or flight bookings. Verify that all necessary parameters are available. If any parameters are missing, ask the user to provide them. If all parameters are complete, call the corresponding function.
    - If the task is not related to hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
      - For flight bookings, output separate outbound and return contents in the order of:
        Departure Airport | Airline | Flight Number | Departure Time | Arrival Airport | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
      - For hotel bookings, output in the order of:
        Property Name | Property Description | Check-in Time | Check-out Time | Prices | Nearby Places | Hotel Class | GPS Coordinates.
    """

    SAVE_AGENT_NAME = "SaveAgent"
    SAVE_AGENT_INSTRUCTIONS = """
    You are a save tool agent. Help me to save the trip plan.
    """

    # Create agent definition
    booking_agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=BOOKING_AGENT_NAME,
        instructions=BOOKING_AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent
    booking_agent = AzureAIAgent(
        client=client,
        definition=booking_agent_definition,
        # Optionally configure polling options
        # polling_options=RunPollingOptions(run_polling_interval=timedelta(seconds=1)),
    )

    # Add the sample plugin to the kernel
    booking_agent.kernel.add_plugin(BookingPlugin(), plugin_name="booking")

    # Create agent definition
    save_agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=SAVE_AGENT_NAME,
        instructions=SAVE_AGENT_INSTRUCTIONS
    )

    # Create the AzureAI Agent
    save_agent = AzureAIAgent(
        client=client,
        definition=save_agent_definition,
    )

    save_agent.kernel.add_plugin(SavePlugin(), plugin_name="saving")

    user_inputs = [
        "I have a business trip from London to New York in Feb 20 2025 to Feb 27 2025 ,help me to book a hotel and fight tickets and save it"
    ]

    chat = AgentGroupChat(
        agents=[booking_agent, save_agent],
        termination_strategy=ApprovalTerminationStrategy(agents=[save_agent], maximum_iterations=10),
    )

    try:
        for user_input in user_inputs:
            # Add the user input as a chat message
            await chat.add_chat_message(
                ChatMessageContent(role=AuthorRole.USER, content=user_input)
            )
            print(f"# User: '{user_input}'")

            async for content in chat.invoke():
                print(f"# {content.role} - {content.name or '*'}: '{content.content}'")

            print(f"# IS COMPLETE: {chat.is_complete}")

            print("*" * 60)
            print("Chat History (In Descending Order):\n")
            async for message in chat.get_chat_messages(agent=save_agent):
                print(f"# {message.role} - {message.name or '*'}: '{message.content}'")
    finally:
        await chat.reset()
        await client.agents.delete_agent(save_agent.id)
        await client.agents.delete_agent(booking_agent.id)


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Disclaimer**:  
Dit document is vertaald met behulp van de AI-vertalingsdienst [Co-op Translator](https://github.com/Azure/co-op-translator). Hoewel wij streven naar nauwkeurigheid, dient u er rekening mee te houden dat automatische vertalingen fouten of onnauwkeurigheden kunnen bevatten. Het oorspronkelijke document in de oorspronkelijke taal dient als de gezaghebbende bron te worden beschouwd. Voor cruciale informatie wordt professionele menselijke vertaling aanbevolen. Wij zijn niet aansprakelijk voor eventuele misverstanden of verkeerde interpretaties die voortvloeien uit het gebruik van deze vertaling.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
